In [1]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_powerdict = pd.read_csv('../data/dictionary/ids.csv').set_index('dictionary_id')

df_powerdict.head(3)

,gppd_idnr,esail_id,name,sett_bmu_id,ngc_bmu_id,4c_offshore_id,windpowernet_id,wikidata_id,wikipedia_id,power_technology_id,eutl_id,eic_id,cfd_id,jrc_id,iaea_id,old_repd_id,new_repd_id
dictionary_id,,,,,,,,,,,,,,,,,
10000,NaN,MARK,Rothes Bio-Plant CHP,"E_MARK-1, E_MARK-2","MARK-1, MARK-2",NaN,NaN,NaN,NaN,NaN,NaN,48W000000MARK-1D,NaN,NaN,NaN,NaN,NaN
10001,"GBR1000377, GBR1000369",DIDC,Didcot,"T_DIDC1, T_DIDC2, T_DIDC4, T_DIDC3, T_DIDC1G, ...","DIDC1, DIDC2, DIDC4, DIDC3, DIDC1G, DIDC2G, DI...",NaN,NaN,NaN,NaN,NaN,97165,"48W00000DIDC01G1, 48W00000DIDC02GZ, 48W00000DI...",NaN,NaN,NaN,NaN,NaN
10002,"GBR1000374, GBR1000375",ABTH,Aberthaw B,"T_ABTH7, T_ABTH8, T_ABTH9, T_ABTH7G, T_ABTH8G,...","ABTH7, ABTH8, ABTH9, ABTH7G, ABTH8G, ABTH9G",NaN,NaN,NaN,NaN,NaN,97175,"48W0000000ABTH7Y, 48W0000000ABTH8W, 48W0000000...",NaN,NaN,NaN,NaN,NaN


In [4]:
df_capture_prices = pd.read_csv('../data/attribute_sources/capture-prices/capture-prices.csv')

df_capture_prices.head()

,ngc_bmu_id,year,capture_price
0,ABRBO-1,2019,39.429610
1,ABRBO-1,2020,32.759415
2,ABRTW-1,2019,39.711074
3,ABRTW-1,2020,32.610071
4,ABTH7,2016,42.271283


In [5]:
df_strike_prices = pd.read_csv('../data/attribute_sources/cfd-strike-prices/cfd-strike-prices.csv')

df_strike_prices.head()

,cfd_id,financial_year,strike_price_GBP_per_MWh
0,AAA-ACH-183,2019,92.80
1,AAA-ACH-183,2020,94.46
2,AAA-ACH-183,2021,95.16
3,AAA-BAD-185,2018,91.94
4,AAA-BAD-185,2019,93.92


In [6]:
def get_reverse_id_map(s_ids):
    external_id_to_dictionary_id = {}
    
    for dictionary_id, external_ids in s_ids.dropna().items():
        for external_id in external_ids.split(', '):
            assert external_id not in external_id_to_dictionary_id.keys(), f'{external_id} has already been assigned to {external_id_to_dictionary_id[external_id]}, cannot be set to {dictionary_id} as well'
            external_id_to_dictionary_id[external_id] = dictionary_id
            
    return external_id_to_dictionary_id

In [12]:
ngc_bmu_to_dictionary_id = get_reverse_id_map(df_powerdict['ngc_bmu_id'])
cfd_to_dictionary_id = get_reverse_id_map(df_powerdict['cfd_id'])

<br>

### BMU to CfD Mapping

In [15]:
# should identify sites with differing CfD prices over multiple contracts
# I'll also need to temporarily use the NG SETT to NGC BMU mapping csv and 
# then the LCCC SETT BMU to CfD contract mapping csv

In [26]:
df_bmu_map = pd.read_csv('../data/attribute_sources/bmu-fuel-types/BMUFuelType 2021-08-09.csv')
ngc_to_sett_bmu_map = df_bmu_map[['NGC_BMU_ID', 'SETT_BMU_ID']].dropna().set_index('NGC_BMU_ID')['SETT_BMU_ID'].to_dict()

df_capture_prices['ngc_bmu_id'].isin(ngc_to_sett_bmu_map.keys()).mean()

0.9151090342679128

In [30]:
df_bmu_to_cfd_map = pd.read_csv('https://www.lowcarboncontracts.uk/data-portal/dataset/10c5ea46-8b0b-497e-ae7f-5ba1c5df8ffe/resource/ab9eafe1-911f-47b7-9f60-bfffa9d87e5a/download/cfd-to-bm-unit-mapping.csv')
cfd_to_sett_bmu_map = df_bmu_to_cfd_map[['CFD_Id', 'BMU_Id']].dropna().set_index('CFD_Id')['BMU_Id'].to_dict()

df_strike_prices['cfd_id'].isin(cfd_to_sett_bmu_map.keys()).mean()

1.0